# Surpriseの使い方

## 概要
Pythonのライブラリ「Surprise」は、推薦システムの開発を効率化するために設計されたライブラリである。本ブログでは、Surpriseの定義や性質、具体的な応用例を数式とPythonのコードを用いて紹介する。これにより、機械学習やAIを学習している学生や企業の研究者が、Surpriseを用いて推薦システムを実装するための基礎を学べる。

## ソースコード

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/gr/02/02_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/gr/02/02_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なりますので注意していただきたい。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行います。

In [ ]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認する。
ついでに乱数のseedの設定をする。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

scipy     : 1.11.2
numpy     : 1.25.2
matplotlib: 3.8.1

Watermark: 2.4.3



## Surpriseの定義と性質

### Surpriseとは
Surpriseは、簡単にカスタマイズ可能であり、ユーザーの好みに応じて推薦システムを構築できるPythonライブラリである。Surpriseは、さまざまなアルゴリズム（例えば、行列分解やk近傍法）をサポートしており、これらを利用して効果的な推薦システムを実装できる。

### Surpriseの特徴
1. **多様なアルゴリズム**: 行列分解、k近傍法、ベースライン推定など、多様な推薦アルゴリズムをサポート。
2. **データセットの柔軟性**: 内蔵のデータセットだけでなく、ユーザーが独自のデータセットを読み込んで利用できる。
3. **評価の簡便さ**: クロスバリデーションやグリッドサーチなど、評価手法が充実している。

## 推薦システムの基礎
推薦システムは、ユーザーとアイテムの間の関係をモデル化するシステムである。主に、協調フィルタリングとコンテンツベースのフィルタリングの二つに大別される。

### 協調フィルタリング
協調フィルタリングは、ユーザーの行動データに基づいて推薦を行う手法である。主に二つのアプローチがある：
- **ユーザーベース協調フィルタリング**: 似たユーザーを見つけ、そのユーザーが評価したアイテムを推薦する。
- **アイテムベース協調フィルタリング**: 似たアイテムを見つけ、そのアイテムを評価したユーザーが評価した他のアイテムを推薦する。

$$
\mathbf{R} =
\begin{bmatrix}
r_{11} & r_{12} & \cdots & r_{1n} \\
r_{21} & r_{22} & \cdots & r_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
r_{m1} & r_{m2} & \cdots & r_{mn} \\
\end{bmatrix}
$$

ここで、$\mathbf{R}$はユーザーとアイテムの評価行列であり、$r_{ij}$はユーザー$i$がアイテム$j$に対して行った評価である。

### コンテンツベースのフィルタリング
コンテンツベースのフィルタリングは、アイテムの属性情報に基づいて推薦を行う手法である。例えば、映画のジャンルや出演者などの情報を利用して、ユーザーの好みに合った映画を推薦する。

## Surpriseの応用例

### 行列分解を用いた推薦システム
行列分解は、評価行列$\mathbf{R}$をユーザー特徴行列$\mathbf{P}$とアイテム特徴行列$\mathbf{Q}$に分解する手法である。これにより、低ランク近似を行い、推薦を行う。

$$
\mathbf{R} \approx \mathbf{P} \mathbf{Q}^T
$$

Surpriseで行列分解を用いる場合、SVD（特異値分解）やNMF（非負値行列因子分解）などのアルゴリズムを利用できる。

#### Pythonでの実装例
以下に、SVDを用いた具体例を示す。




In [ ]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

# データの読み込み
data = Dataset.load_builtin("ml-100k")
trainset, testset = train_test_split(data, test_size=0.25)

# SVDアルゴリズムの適用
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# 精度の評価
accuracy.rmse(predictions)

In [ ]:
### k近傍法を用いた推薦システム
k近傍法（k-NN）は、ユーザーやアイテムの類似度に基づいて推薦を行う手法である。Surpriseでは、ユーザーベースやアイテムベースのk-NNを実装できる。

#### Pythonでの実装例
以下に、アイテムベースのk-NNを用いた具体例を示す。



In [ ]:
from surprise import KNNBasic

# アイテムベースのk-NNアルゴリズムの適用
algo = KNNBasic(k=40, sim_options={"user_based": False})
algo.fit(trainset)
predictions = algo.test(testset)

# 精度の評価
accuracy.rmse(predictions)

## メリットとデメリット

### メリット
1. **高い柔軟性**: 複数のアルゴリズムを容易に試すことができる。
2. **評価手法の充実**: クロスバリデーションやグリッドサーチを利用して、モデルの評価と最適化が可能。
3. **豊富なドキュメント**: ドキュメントが充実しており、初学者でも学びやすい。

### デメリット
1. **学習曲線**: 初学者にとっては、さまざまなアルゴリズムや評価手法の理解に時間がかかる場合がある。
2. **パフォーマンス**: 大規模データセットに対しては、計算リソースが多く必要となる場合がある。

## 結論
この記事では、PythonのSurpriseライブラリについて、その定義、性質、具体的な応用例を紹介した。Surpriseを利用することで、推薦システムの開発が容易になり、多様なアルゴリズムを試すことができる。特に、行列分解やk近傍法を用いた実装例を示し、実際のコードを通じてその利用方法を学んだ。

## 参考文献
- Surprise Documentation: [https://surprise.readthedocs.io/en/stable/](https://surprise.readthedocs.io/en/stable/)
- Matrix Factorization Techniques for Recommender Systems: [https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf](https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf)
- Wikipedia: [https://ja.wikipedia.org/wiki/推薦システム](https://ja.wikipedia.org/wiki/推薦システム)